In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
path = "../data/Posts.xml"
posts = pd.read_xml(path)

In [3]:
posts.shape

(244066, 22)

In [4]:
posts.sample(1)

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,...,Tags,AnswerCount,CommentCount,ClosedDate,CommunityOwnedDate,ContentLicense,ParentId,OwnerDisplayName,LastEditorDisplayName,FavoriteCount
14927,36932,2,NaN,2011-01-14T20:37:33.427,6,NaN,<p>Ask them to defend agile methodologies. And...,3119.0,NaN,None,...,None,NaN,9,None,2011-01-16T03:02:08.853,CC BY-SA 2.5,36925.0,None,None,NaN


In [5]:
posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score',
       'ViewCount', 'Body', 'OwnerUserId', 'LastEditorUserId', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'ClosedDate', 'CommunityOwnedDate', 'ContentLicense', 'ParentId',
       'OwnerDisplayName', 'LastEditorDisplayName', 'FavoriteCount'],
      dtype='object')

The column ```Body``` is a raw HTML code. Before applying any model, we need to clean it from redundant tags.

For this, we will use a library ```BeautifulSoup```

In [6]:
posts = posts.dropna(subset=["Body"], axis=0) # next cell won't work otherwise

In [7]:
from bs4 import BeautifulSoup

def html_to_str(row_html: str) -> str:
    soup = BeautifulSoup(row_html, "html.parser")
    return soup.get_text(separator=' ')

posts["Body"] = posts["Body"].apply(html_to_str)

Most columns are of no value. We will keep only those that are potentilly useful.

In [8]:
columns_to_keep = ["Body", "ViewCount", "CreationDate", "PostTypeId", "Score", "Tags", "Title"]
posts = posts[columns_to_keep]

In [9]:
# posts = posts[posts.CreationDate >= ...] # maybe useful
questions = posts[posts.PostTypeId == 1]
answers = posts[posts.PostTypeId == 2]

### Predict question ```Score``` based on the ```Body```'s embedding

In [10]:
questions_subset = questions.copy()[:10_000]

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sentence_transformers import SentenceTransformer

import numpy as np

In [13]:
from tqdm import tqdm
tqdm.pandas()

In [14]:
# Embedder models
models = {
    'Albert': 'paraphrase-albert-small-v2',
    'Roberta': 'all-distilroberta-v1',
    'DistilBert': 'multi-qa-distilbert-cos-v1',
    'MiniLM1': 'all-MiniLM-L6-v2',
    'MiniLM2': 'all-MiniLM-L12-v2',
    'MiniLM3': 'paraphrase-MiniLM-L3-v2'
}

In [15]:
def estimate_embedder(model_name: str) -> float:
    """
    Estimates the performance of a linear regression model using embeddings
    generated by a specified SentenceTransformer model from Hugging Face.

    Args:
        model_name (str): The name of the model to be used for generating embeddings.

    Returns:
        float: The Mean Absolute Error (MAE) of the linear regression model
               on the test set.
    """
    # Download a model from Hugging Face using its name
    selected_model = models[model_name]
    embedder = SentenceTransformer(selected_model)

    bodies = questions_subset["Body"].tolist()

    X = []
    for body in tqdm(bodies, desc=f"Encoding posts with {model_name}"):
        # Encode each "body" and append it to X
        encoded_body = embedder.encode(body)
        X.append(encoded_body)

    X = np.array(X)
    y = questions_subset["Score"].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1200)

    # Create and fit a simple linear regression model
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    # Evaluate model performance
    MAE = mean_absolute_error(y_test, y_pred)
    return MAE

In [16]:
models_mae = []
for model in models.keys():
    mae = round(estimate_embedder(model_name=model), 3)
    models_mae.append({"model": model, "mae": mae})

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Encoding posts: 100%|██████████| 10000/10000 [00:59<00:00, 167.36it/s]


In [19]:
for model_mae in models_mae:
    print(f"MODEL={model_mae['model']}, MAE={model_mae['mae']}")

# MODEL=Albert, MAE=5.392
# MODEL=Roberta, MAE=5.219
# MODEL=DistilBert, MAE=5.2
# MODEL=MiniLM1, MAE=5.229
# MODEL=MiniLM3, MAE=4.857
# MODEL=MiniLM4, MAE=5.053

MODEL=Albert, MAE=21.199
MODEL=Roberta, MAE=20.627
MODEL=DistilBert, MAE=20.812
MODEL=MiniLM1, MAE=19.278
MODEL=MiniLM2, MAE=18.827
MODEL=MiniLM3, MAE=21.884
